<a href="https://colab.research.google.com/github/lisaong/diec/blob/master/day3/edge_time_series_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Edge Device Model Deployment

In this notebook, we will train a deep learning model and deploy it to the edge.

1. Train model using Keras
2. Convert to compressed floating point version using Tensorflow Lite
3. Deploy on Raspberry Pi

We will use this dataset, which is gathered from a smart factory in Germany: https://www.kaggle.com/inIT-OWL/versatileproductionsystem



In [0]:
%tensorflow_version 2.x

In [0]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv1D, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint

sns.set(style='whitegrid')
%matplotlib inline

## Dataset

Load the dataset from our github repository.

In [0]:
!curl https://raw.githubusercontent.com/lisaong/diec/master/data/versatileproductionsystem.zip --output data.zip

In [0]:
!unzip -o data.zip

In [0]:
# read one of the data files
df = pd.read_csv('./Filling_ALL.module.csv', index_col=0)
df.head()

## Task Definition

A common factory monitoring scenario is to predict a target based on past history.

For the purposes of this workshop, let's try to predict the value of 'I_BottlesReserveAvailable', using last 10 values of each feature.

In [0]:
# Target
target = 'I_BottlesReserveAvailable'

df[target].plot();

In [0]:
# check for balanced classes
df[target].value_counts()

In [0]:
# drop some columns that the target is derived from (otherwise we are cheating)
df.drop(['BottlesAvailable', 'BottlesNotUsed', 'I_BottleStorageFull'], axis=1, inplace=True)
df.columns

## Reserve for testing

Reserve the some values for testing on the Raspberry Pi.


In [0]:
test_size = 100

df_train = df.iloc[:-test_size]
df_test = df.iloc[-test_size:]

df_train.shape, df_test.shape

In [0]:
# save input numpy arrays for later use
pickle.dump(df_train.loc[:, df_train.columns != target].values,
            open('./X_train.pkl', 'wb'))

pickle.dump(df_test.loc[:, df_test.columns != target].values,
            open('./X_test.pkl', 'wb'))

## Feature Selection

First, some basic feature selection: Filter out columns that don't change very much (low variance). 

In [0]:
# Candidate features

candidates = df_train.columns[df_train.columns != target]
candidates

In [0]:
# Apply variance threshold of 0.1
vt = VarianceThreshold(threshold=0.1)
vt.fit(df_train[candidates])

# filtered out features
features = candidates[vt.get_support()]
features

## Scaling

Next, scale the features so that we can speed up training.

In [0]:
feature_scaler = MinMaxScaler(feature_range=(-1, 1))
feature_values = feature_scaler.fit_transform(df_train[features])
feature_values

## Time Series Windowing

Next, convert the time series so that each entry contains a series of timesteps.

Before: rows, features

After: rows, timesteps, features

Note that some rows will be removed because we are taking a window of values.

In [0]:
timesteps = 10

print(feature_values.shape) # (rows, features)

rolling_indexes = [(range(i, i+timesteps))
                   for i in range(feature_values.shape[0]-timesteps)]

X_sequence = np.take(feature_values, rolling_indexes, axis=0)
X_sequence.shape # (rows, timesteps, features)

In [0]:
# shift y down by 1 value so that we are predicting the future
# make sure y is the same length as X_sequence

print(df[target].shape)
y = df[target][1:X_sequence.shape[0]+1].values
y.shape

## Train model

We will train a very simple 1D convolutional model to try to predict y from X_sequence. 

To recap:

- X_sequence (rows, timesteps, features): each scaled feature at [t-9] ... [t]
- y (rows): target[t+1]

The convolutional 1D layer will look at windows of timesteps, one feature at a time. Here's a more detailed reference on it for time series data: https://missinglink.ai/guides/keras/keras-conv1d-working-1d-convolutional-neural-networks-keras/


Note: we can also train an LSTM model, but currently Tensorflow Lite with Keras does not support it nicely (yet): https://github.com/tensorflow/tensorflow/issues/30864

In [0]:
# create training and validation set
X_train, X_val, y_train, y_val = train_test_split(X_sequence, y, test_size=0.1,
                                                  stratify=y)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

In [0]:
model = Sequential()

# input_shape=(timesteps, features)
model.add(Conv1D(64, kernel_size=3,
                 input_shape=(X_train.shape[1], X_train.shape[2]),
                 activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [0]:
mc = ModelCheckpoint('./cnn.h5', save_best_only=True)

In [0]:
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_val, y_val), callbacks=[mc])

In [0]:
fig, ax = plt.subplots(figsize=(10, 8))
ax.plot(history.history['loss'], label='train')
ax.plot(history.history['val_loss'], label='val')
ax.legend()
ax.set_title('Learning curve')
ax.set_xlabel('Epochs')
ax.set_ylabel('Loss')
plt.show()

### Predictions

We'll test the model by getting predictions for our test data.

The steps followed here will be repeated on the Raspberry Pi, so we want to make sure these are repeatable.

In [0]:
df_test_X = df_test.loc[:, df_test.columns != target]
df_test_y = df_test[[target]]

df_test_X.shape, df_test_y.shape

In [0]:
# select features
test_X = vt.transform(df_test_X)

# scale
test_X_scaled = feature_scaler.transform(test_X)

# create windowed
rolling_indexes = [(range(i, i+timesteps))
                   for i in range(test_X_scaled.shape[0]-timesteps)]

test_X_sequence = np.take(test_X_scaled, rolling_indexes, axis=0)

print(test_X_sequence.shape) # (rows, timesteps, features)

In [0]:
# shift y down by 1 value so that we are predicting the future
# make sure y is the same length as test_X_sequence

test_y = df_test_y[1:test_X_sequence.shape[0]+1] 
print(test_y.shape)

In [0]:
# predict by loading the checkpointed model with the lowest validation loss
best_model = load_model('./cnn.h5')
pred = best_model.predict_classes(test_X_sequence)

In [0]:
# score
print(classification_report(test_y, pred))
print(confusion_matrix(test_y, pred))

In [0]:
# try to predict some class 0 values using validation set

pred_val = model.predict_classes(X_val)

print(classification_report(y_val, pred_val))
print(confusion_matrix(y_val, pred_val))

## Deployment

1. Save pre-processors
2. Convert to Tensorflow Lite: https://www.tensorflow.org/lite/convert#from_model_training_to_device_deployment

In [0]:
# Save preprocessors, create a dictionary so that everything in 1 place

preprocessors = {
    'variance_threshold' : vt,
    'feature_scaler' : feature_scaler   
}

pickle.dump(preprocessors, open('./preprocessors.pkl', 'wb'))

In [0]:
# Convert to Tensorflow lite
saved_model = tf.keras.models.load_model('./cnn.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(saved_model)

# https://www.tensorflow.org/api_docs/python/tf/lite/Optimize
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_LATENCY] 

tflite_model = converter.convert()
with open('./cnn.tflite', 'wb') as f:
  f.write(tflite_model)

### Testing TensorFlow Lite Inference

https://www.tensorflow.org/lite/guide/inference#running_a_model

Before running inference on a Raspberry Pi, it is easier to write code and test out the inference on a PC or Google Cloud.

In [0]:
# This is the code we will run on the Raspberry Pi
import pickle
import numpy as np
import tensorflow as tf

def load_test_data(data_filename, preprocessors_filename):
  timesteps = 10

  # Load test data and process it
  test_array = pickle.load(open(data_filename, 'rb'))

  preprocessors = pickle.load(open(preprocessors_filename, 'rb'))
  vt = preprocessors['variance_threshold']
  feature_scaler = preprocessors['feature_scaler']

  # select features
  test_X = vt.transform(test_array)

  # scale
  test_X_scaled = feature_scaler.transform(test_X)

  # create windowed
  rolling_indexes = [(range(i, i+timesteps))
                     for i in range(test_X_scaled.shape[0]-timesteps)]

  test_X_sequence = np.take(test_X_scaled, rolling_indexes, axis=0)
  print(test_X_sequence.shape) # (rows, timesteps, features)
    
  return test_X_sequence

In [0]:
X_test = load_test_data('./X_test.pkl', './preprocessors.pkl')

In [0]:
def predict(model_path, test_data):
  # Load TFLite model and allocate tensors.
  interpreter = tf.lite.Interpreter(model_path=model_path)
  interpreter.allocate_tensors()

  # Get input and output tensors.
  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details()

  # Test model on input data.
  input_shape = input_details[0]['shape']

  # Loop through each row of test_data and perform inference
  for i in range(test_data.shape[0]):

    # add batch dimension
    input_data = np.expand_dims(test_data[i], axis=0).astype('float32')

    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()

    # The function `get_tensor()` returns a copy of the tensor data.
    # Use `tensor()` in order to get a pointer to the tensor.
    output_data = interpreter.get_tensor(output_details[0]['index'])
    print(output_data)

### Exercise 1: Testing the compressed model
<p>
<font color="red">
Update the cell below to get predictions from N randomly chosen samples, using the Tensorflow Lite version of the model.
</font>
</p>
<p>
<font color="green">
Submission:
<ol>
<li type='a'>Paste the updated code into the submission worksheet</li>
<li type='a'>Paste predictions into the submission worksheet</li>
</ol>
</font>
</p>

In [0]:
# Exercise: test out your compressed tensorflow lite model to get
# predictions from randomly selected samples before deploying to the Edge device.
#
# Hint1: you may use the helper function above to get a prediction.
# Hint2: you may use the following code to get a random selection of samples
# N = <pick_a_number>
# random_index = np.random.choice(range(len(X_test)), N)
#

X_test = load_test_data('./X_test.pkl', './preprocessors.pkl')

#
# ADD YOUR CODE BELOW
#